In [344]:
import pandas as pd
import numpy as np
import openpyxl

from credit_card import CreditCard
from person import Person, Expense
from recommender import Recommender
from maximizer import Maximizer
from globals import global_vars

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [345]:
global_vars.setBonusToggle(False)
years = 3

In [346]:
credit_card_df = pd.read_excel("credit_cards.xlsx")
credit_card_df.head()

,Card,Ignore,Company,Annual Fee,Fee Free Months,Point Type,Point to Dollar Ratio,Point to Dollar Ratio (Low),Point to Dollar Ratio (High),Opening Bonus Points,Opening Bonus Spending Threshold,Opening Bonus Months Limit,DefaultMultiplier,PointMultiplier1,PointMultiplier2,PointMultiplier3,PointMultiplier4,PointMultiplier5,PointMultiplier6
0,Platinum Card,No,American Express,695,0,Amex Point,0.0100,0.08,0.015,80000,6000,6,1.00,"Airfare,5,500000","Hotels,5",NaN,NaN,NaN,NaN
1,Gold Card,No,American Express,250,0,Amex Point,0.0100,0.08,0.015,60000,4000,6,1.00,"Dining,4","Groceries,4,25000","Airfare,3",NaN,NaN,NaN
2,Blue Cash Preferred,No,American Express,95,12,Cash,1.0000,1.00,1.000,250,6000,6,0.01,"Groceries,0.06,6000","Streaming,0.06","Transit,0.03","Gas,0.03",NaN,NaN
3,Green Card,No,American Express,150,0,Amex Point,0.0100,0.08,0.015,60000,3000,6,1.00,"Airfare,3","Hotels,3","Transit,3","Dining,3",NaN,NaN
4,Delta SkyMiles Platinum,No,American Express,250,0,Delta SkyMiles,0.0125,0.01,0.015,50000,3000,6,1.00,"Airfare,3","Hotels,3","Dining,2","Groceries,2",NaN,NaN


In [347]:
def createCardObject(card_row):
    if card_row["Ignore"] == "No":
        # Take a row from the credit card table and transform that row to the CreditCard object. Return that object.
        card = CreditCard(name=card_row["Card"], comp=card_row["Company"], ratio=card_row["Point to Dollar Ratio"], default_point_multiplier=card_row["DefaultMultiplier"])
        card.setAnnualFee(amount=card_row["Annual Fee"], free_months=card_row["Fee Free Months"])
        card.setOpeningBonus(points=card_row["Opening Bonus Points"], spending_criteria=card_row["Opening Bonus Spending Threshold"], months_limit=card_row["Opening Bonus Months Limit"])
        for cell in card_row.filter(regex="PointMultiplier").dropna():
            # The three values are stored in the column as a string and delimited with a column
            # The values are 'category,multiplier,limit'. If the limit is omitted, assume it does not exist.
            values = cell.split(",")
            if len(values) == 3:
                card.addPointsMultiplier(category=values[0], multiplier=float(values[1]), limit=float(values[2]))
            else:
                card.addPointsMultiplier(category=values[0], multiplier=float(values[1]))
        return card
    else:
        return None

card_list = []
for i, row in credit_card_df.iterrows():
    card = createCardObject(row)
    if card is not None:
        card_list.append(card)


In [348]:
expense_df = pd.read_excel("expenses_jason_gaby.xlsx")
expense_df.head()

,Category,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,Gas,200,200,200,200,200,200,200,200,200,200,200,200,2400
1,Transit,100,100,100,100,100,100,100,100,100,100,100,100,1200
2,Streaming,50,50,50,50,50,50,50,50,50,50,50,50,600
3,Groceries,800,800,800,800,800,800,800,800,800,800,800,800,9600
4,Dining,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,1500,18000


In [349]:
def addExpensesFromDataFrame(df, person):
    for i, row in df.iterrows():
        category = row["Category"]
        amounts = row[["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]].to_list()
        expense = Expense(category, amounts)
        person.addExpense(expense)
    return

jason = Person("Jason", "Gaby")
addExpensesFromDataFrame(expense_df, jason)

In [350]:
jason.getExpenses()[-1].getAmountOverMonthRange()

174000

In [351]:
recommender = Recommender(jason, card_list, years=years)
sorted_recommendation = recommender.recommendation_df.sort_values(by=["Net Value ($)"], ascending=False)
print(sorted_recommendation)



                       Card  Net Benefit ($)  Net Costs ($)  Net Value ($)
9          Sapphire Reserve         22612.50           1650       20962.50
12        Venture X Rewards         20286.00           1185       19101.00
10        Freedom Unlimited         15934.50              0       15934.50
11          Venture Rewards         16146.00            285       15861.00
8        Sapphire Preferred         16098.75            285       15813.75
14              Double Cash         14958.00              0       14958.00
4   Delta SkyMiles Platinum         13083.75            750       12333.75
13                  Premier         11439.00            285       11154.00
1                 Gold Card         11403.00            750       10653.00
3                Green Card         10791.00            450       10341.00
6       Delta SkyMiles Gold         10383.75            198       10185.75
0             Platinum Card         11799.00           2085        9714.00
7    Delta Skymiles Reser

In [352]:
maximizer = Maximizer(jason, card_list, years=years)
recommendations = maximizer.getRecommendations()
for key in recommendations:
    print(f"{key}: {recommendations[key]['card'].getName()} - ${recommendations[key]['value']: .2f}")


Hotels: Sapphire Reserve - $ 3750.00
Car Rentals: Sapphire Reserve - $ 540.00
Online Retail: Blue Cash Everyday - $ 540.00
Drug Store: Double Cash - $ 18.00
Other: Double Cash - $ 10440.00
Gas: Blue Cash Everyday - $ 216.00
Transit: Sapphire Reserve - $ 162.00
Streaming: Double Cash - $ 36.00
Groceries: Blue Cash Preferred - $ 1538.00
Dining: Sapphire Reserve - $ 2430.00
Airfare: Sapphire Reserve - $ 5400.00


In [353]:
print(f"\nRecommended Card: {recommender.getRecommendationName()}")
print(f"Net Value: ${recommender.getRecommendationNetValue():.2f}")

print(f"\nMaximum Possible Net Value if optimizing card usage: ${maximizer.getRecommendationValue():.2f}")
print(f"Possible additional value: ${maximizer.getRecommendationValue() - recommender.getRecommendationNetValue():.2f}")


Recommended Card: Sapphire Reserve
Net Value: $20962.50

Maximum Possible Net Value if optimizing card usage: $25070.00
Possible additional value: $4107.50


In [354]:
# DO NOT DELETE!
amount = 1_000
n_samples = 2
n_dimensions = 3
v = np.linspace(0, amount, n_samples)
sum = np.array(np.meshgrid(*[v] * n_dimensions)).sum(axis=0)
print(sum)



[[[   0. 1000.]
  [1000. 2000.]]

 [[1000. 2000.]
  [2000. 3000.]]]
